In [1]:
from corenlp_xml_reader import AnnotatedText as A
from lxml import html
from collections import defaultdict
# import networkx as nx
# import nxpd
from wordcloud import WordCloud
from utils.file_io_helper import unicode_normalizer,read_books_json
import json
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import OrderedDict
import glob
import os 
%matplotlib inline
plt.rcParams["figure.figsize"] = (10,6)


In [25]:
def sentence_out(sent):
    return ' '.join([w['word'] for w in sent['tokens']])


def co_occurence(doc,sentiment):
    pair_sent = defaultdict(list)
    for sentence in doc.sentences:

        curr_name = []
        prev_ner = None
        for ent in sentence['tokens']:
            if ent['ner'] == 'PERSON':
                name = ent['word']
                size = len(name)
                if not prev_ner or prev_ner == name:
                    prev_ner = name
                else:
                    prev_ner += ' ' + name
            else:
                if prev_ner:
                    curr_name.append(prev_ner)
                prev_ner = None
        if prev_ner:
            curr_name.append(prev_ner)
            
        pair = []
        if curr_name:
            prev = None
            for name_ in curr_name:
                if (not prev or prev!=name_) and len(pair)<2:
                    pair.append(name_)
                    prev = name_
            pair = tuple(sorted(pair))
            
        if len(pair)==2:
            pair_sent[pair].append((sentence_out(sentence),sentiment[sentence['id']]))
    return pair_sent

def xml_analysis(xml_file,limit = 1):
    translate = {'Verynegative':-2, 'Negative':-1,'Positive':1,'Verypositive':2,'Neutral':0}
    xml = open(xml_file).read()
    annotated_text = A(xml)
    
    tree = html.fromstring(xml)
    sentiment = tree.xpath('//sentence/@sentiment')
    sentiment = [translate[key] for key in sentiment]

    pair_dict =co_occurence(annotated_text,sentiment)
    
    avg_sent = lambda x:np.mean([y[1] for y in x])
    return pair_dict

def combine_one_xml(f):
    related_sentence_dict = {}
    related_sentence_dict[str(os.path.basename(f).split('.')[0])]=xml_analysis(f)
    return related_sentence_dict

def unpack(lst):
    return ' '.join([i[0] for i in lst]), [i[1] for i in lst]      
    

In [6]:
related_sent = combine_one_xml('../data/full-text-tag/1.1984_full.txt.xml')

In [15]:
len(related_sent['1'].keys())

47

In [49]:
clean_p={k:unpack(v) for k,v in related_sent['1'].iteritems()} 
clean_p = [p for p in clean_p if len(p[1])>=7]
# clean_p = [pp for p in clean_p for pp in p]
# clean_p= [p for p in clean_p if len(p[1]>5)]
# docs,sentiments,relations = zip(*clean_p)
# relations = [r[0] for r in relations]
# docs = list(docs)


In [50]:
len(clean_p)

36

In [53]:
clean_p

[(u'Winston', u'Winston KNEW'),
 (u'Goldstein', u'Parsons'),
 (u'Tillotson', u'Winston'),
 (u'Parsons', u'Winston'),
 (u'Mrs Parsons', u'Parsons'),
 (u'Ogilvy', u'Winston'),
 (u'Julia', u'Winston'),
 (u'B.B.', u'Parsons'),
 (u'Rutherford', u'Winston'),
 (u'Charrington', u"O'Brien"),
 (u'Chaucer', u'Shakespeare'),
 (u'Mrs Parsons', u'Winston'),
 (u'Charrington', u'Winston'),
 (u'Julia', u"O'Brien"),
 (u'Aaronson', u'Winston'),
 (u'Brother', u"O'Brien"),
 (u'B.B.', u'Winston'),
 (u'Smith', u'Winston'),
 (u'Norman', u'Winston'),
 (u'Syme', u'Winston'),
 (u'Martin', u'Winston'),
 (u'Julia', u'Katharine'),
 (u'Charrington', u'Clement'),
 (u'Julia the story of Winston', u'Winston'),
 (u"O'Brien", u'Winston'),
 (u'Saint Sebastian', u'Winston'),
 (u'Katharine', u'Winston'),
 (u'Winston', u'Withers'),
 (u'Winston', u'Winston/Smith/Winston Smith'),
 (u'Goldstein', u'Winston'),
 (u'Julia Winston', u'Katharine'),
 (u'Goldstein', u"O'Brien"),
 (u'Smith', u'Winston Smith'),
 (u'Martin', u"O'Brien"),